-Constraint to set one node to 1 for the ending node: $(1-\sum_i y_i + \sum_{i\neq j} 2y_iy_j)$

-Constraint to set other nodes to 0 or 2: $(\sum_jy_j)(2-\sum_jy_j)^2= \sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq j}y_iy_jy_k$. Adding ancilla variables to make it QUBO:
$\sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq k}[w_{i,j}y_k]+6\sum_{i\neq j,i\neq k,j\neq k}[y_iy_j-2w_{i,j}(y_i+y_j)+3w_{i,j}]$

-Constraint to set one node to $n$ for the starting node: $(\sum_iy_i-n)^2=n^2-(2n-1)\sum_i y_i+\sum_{i\neq j} 2y_iy_j$

-Minimum distance objective function:  $w\sum_{i \notin start, i \notin start} y_i$ if $w$ is not there 0 would be a viable solution too. While the constraints only add a penalty or reward of 1/-1, the objective function's penalty grows linearly with the routs length. Therefore, it is imperetive that we nulify the effect of this growth with a small weight. A weight too big wont do shit and a weight too small will make the adiabatic gap very tiny and inceases the error rate. 

In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
#import the necessary shit
import matplotlib.pyplot as plt
import numpy as np
import skopt
import skopt.space as space

import dwave_networkx as dnx
import networkx as nx
import minorminer
import neal
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import dimod

from qpr.routing import NetList, Route, get_qubo
from qpr.quantum_utils import find_embedding_minorminer

%matplotlib inline

In [ ]:
edges = [
    (6, 7), (7, 8), (6, 3), (7, 4), (8, 5),
    (3, 4), (4, 5), (3, 0), (4, 1), (5, 2),
    (0, 1), (1, 2),
]
rev_edges = [(j, i) for i, j in edges]
edges = edges + rev_edges
edges = edges + [
    ('o0', 0),  # (0, 'i0'), (1, 'i1'), ('o1', 1),
    # (2, 'i2'), ('o2', 2), (3, 'i3'), ('o3', 3),
    # (5, 'i5'), ('o5', 5), (6, 'i6'), ('o6', 6),
    (7, 'i7'),  # ('o7', 7), (8, 'i8'), ('o8', 8),
]

# cartesian frame, i.e. (x, y) pairs, with origin at bottom left
pos = {
    0: (0, 0), 1: (1, 0), 2: (2, 0), 3: (0, 1),
    4: (1, 1), 5: (2, 1), 6: (0, 2), 7: (1, 2),
    8: (2, 2), 'o0': (-.8, -.4),  # 'i0': (-.4, -.8),
    # 'i1': (.8, -1), 'o1': (1.2, -1), 
    # 'i2': (2.6, -.4), 'o2': (2.4, -.6),
    # 'i3': (-1, .8), 'o3': (-1, 1.2),
    # 'i5': (3, .8), 'o5': (3, 1.2),
    # 'i6': (-.4, 2.6), 'o6': (-.6, 2.4),
    'i7': (.8, 3),  # 'o7': (1.2, 3),
    # 'i8': (2.4, 2.6), 'o8': (2.6, 2.4),
}
super_graph = nx.DiGraph(edges)
nx.draw_networkx(super_graph, pos=pos)

In [ ]:
route_obj = Route(
    super_graph,
    NetList([('s1', ['t11']), ]),
    set(),
    dict(s1='o0', t11='i7'),
    pos
)
route_obj.draw_arch()

In [ ]:
Q = get_qubo(route_obj.arch_graph, route_obj.netlist)

In [ ]:
%%time

dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
embedding, chain_len = find_embedding_minorminer(Q, A, num_tries=5)
# the shortest chain_len I've seen with num_tries=1000 is 5
# (SP: takes 2.5 mins on my machine)
display(chain_len)

In [ ]:
connectivity_structure = dnx.chimera_graph(16,16)
fig=plt.figure(figsize=(25, 25))
dnx.draw_chimera_embedding(connectivity_structure, embedding)

# QPU

In [ ]:
#set up the Qubo
def optimize_qannealer(sampler, Q):
    response = sampler.sample_qubo(
        Q, chain_strength=4, annealing_time=500, auto_scale=True, num_reads=1000
    )
    return response


fixed_sampler = FixedEmbeddingComposite(
    DWaveSampler(solver={'lower_noise': True, 'qpu': True}), embedding
)
q_response = optimize_qannealer(fixed_sampler, Q)

In [ ]:
display(q_response.first)
best_q_answer = q_response.first.sample

In [ ]:
qv2np = {f'y{i}': edge for i, edge in enumerate(route_obj.arch_graph.edges)}
def qubo_answer2node_pairs(ans):
    # answer is a dict of {qubo var name: 0/1} e.g. {'y0': 0, 'y1': 1, etc}
    # it can have auxiliary variables not found in self.np2qv or self.qv2np
    return [qv2np[var_name] for var_name in qv2np if ans[var_name] == 1]

In [ ]:
# G.draw()

# edge_set = G.qubo_answer2node_pairs(q_response.samples()[1])
# G.highlight_edge_list(edge_set)

fig, ax = plt.subplots()
route_obj.edge_list = qubo_answer2node_pairs(best_q_answer)
route_obj.draw(draw_netlist=False, draw_subgraph=False, axes=[ax, None, None])
ax.set_title(f'{route_obj.score()}');

In [ ]:
def make_ax_grid(n, ax_h=4, ax_w=6, ncols=4):
    nrows = int(np.ceil(n / ncols))
    fig_h = nrows * ax_h
    fig_w = ncols * ax_w
    return plt.subplots(nrows=nrows, ncols=ncols, figsize=(fig_w, fig_h))

# Exact solver

In [ ]:
len(Q) # this shouldn't be 752, something is probably broken..

In [ ]:
%%time

# TODO this eats up the RAM if Q is large
# run at your own risk, open the system monitor or something
# and keep an eye on the memory usage..
assert len(Q) < 30

exact_response = dimod.ExactSolver().sample_qubo(Q)
display(exact_response.record)

In [ ]:
# .data() sorts by energy by defaults but returns an iterator (not a SampleSet)
# the iterator yields a named tuple
# .samples(n) sort by energy, take at most n samples, return a SampleArray
# which is a view, mapping the var names to the values (i.e returns dicts), It is
# indexable i.e. .samples()[10] works
# .record returns record array of Sample objects which is basically a 
# numpy-scliceable list of named tuples (samples). Also .record.energy
# returns a numpy array of energies, .record.samples returns a 2d numpy
# array of qubo answers etc.
# Iterating over the SampleSet, calls .samples() internally, i.e. it gets sorted
# .first calls data() internally so it does the sorting anyway!

# This function returns all the min energy solutions as a list of {var name: val} dicts
def get_all_min_energy(sample_set):
    min_energy = np.min(sample_set.record.energy)
    # use .record since it is slicing friendly, this returns a 2d-array-like recarray
    records = sample_set.record[sample_set.record.energy == min_energy]
    # make dicts out of each answer using the original var names (i.e. sample_set.variables)
    return [dict(zip(sample_set.variables, i.sample)) for i in records], min_energy

In [ ]:
min_energy_sols, _ = get_all_min_energy(exact_response)
fig, axes = make_ax_grid(len(min_energy_sols))
display(len(min_energy_sols))

for ax, answer_dict in zip(axes.flat, min_energy_sols):
    G.draw(edge_labs=False, ax=ax)  # edge_labs=False)

    edge_set = G.qubo_answer2node_pairs(answer_dict)
    G.highlight_edge_list(edge_set, ax=ax)
fig.tight_layout()

# Simulated Annealing

In [ ]:
%%time



Q2 = get_qubo(route_obj.arch_graph, route_obj.netlist, w_and=2)
sa_response = neal.SimulatedAnnealingSampler().sample_qubo(Q2, num_reads=1000, seed=1234).aggregate()
# display(sa_response)

# edge_set = G.qubo_answer2node_pairs(sa_response.first.sample)
# G.draw(edge_labs=False)
# G.highlight_edge_list(edge_set)

min_energy_sols, _ = get_all_min_energy(sa_response)
fig, axes = make_ax_grid(len(min_energy_sols[:20]))
display(len(min_energy_sols))

for ax, answer_dict in zip(axes.flat, min_energy_sols):
    route_obj.edge_list = qubo_answer2node_pairs(answer_dict)
    route_obj.draw(draw_netlist=False, draw_subgraph=False, axes=[ax, None, None])
    ax.set_title(f'{route_obj.score()}')
fig.tight_layout()

In [ ]:
def sim_anneal(w1=0, w2=1, w3=1, w4=1, w_and=6):
    Q2 = get_qubo(route_obj.arch_graph, route_obj.netlist)
    result = neal.SimulatedAnnealingSampler().sample_qubo(
        Q, num_reads=1000, seed=1234
    ).first.sample
    route_obj.edge_list = qubo_answer2node_pairs(result)
    score = route_obj.score()
    if score == np.inf:
        score = 100000000
    
    return score

sim_anneal(w_and=2)

In [ ]:
### stick to uniform priors and linear exp to avoid known skopt bugs..sigh
search_space = [
    space.Real(0, 10, prior='uniform', name='w1'),
    space.Real(0, 10, prior='uniform', name='w2'),
    space.Real(0, 10, prior='uniform', name='w3'),
    space.Real(0, 10, prior='uniform', name='w4'),
    space.Real(0, 10, prior='uniform', name='w_and'),
]

res_gp = skopt.gp_minimize(
    sim_anneal, search_space, acq_func='gp_hedge', n_calls=500,
    n_random_starts=400, random_state=2, verbose=True,
)
res_gp.x, res_gp.fun

In [ ]:
gp_best_func_val = np.array(
    [np.min(res_gp.func_vals[:i]) for i in range(1, res_gp.func_vals.size+1)]
)
plt.plot(gp_best_func_val, label='GP best')
plt.scatter(np.arange(res_gp.func_vals.size), res_gp.func_vals, label='GP eval')
plt.xlabel('Num eval')
plt.ylabel('score')
plt.legend();

# hybrid solution

In [ ]:
import hybrid

# Construct a problem
offset=0.0
#vartype = dimod.BINARY
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset)

# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

# Solve the problem
init_state = hybrid.State.from_problem(bqm)
final_state = workflow.run(init_state).result()

# Print results
print("Solution: sample={.samples.first}".format(final_state))

In [ ]:
G.draw(edge_labs=False)  # edge_labs=False)

edge_set = G.qubo_answer2node_pairs(answer_dict)
G.highlight_edge_list(edge_set)
